In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from matplotlib import gridspec 
import random
import xgboost as xgb
from sklearn import preprocessing
from math import sqrt
train1 = pd.read_csv("train.csv")
test1 = pd.read_csv("test.csv")
top = np.percentile(train1.Purchase,99.5)
#sns.boxplot(x=train1[ (train1['Purchase']<top)].Purchase)
train1 = train1[ (train1['Purchase']<top) ]
test1['Purchase']=test1.index
user_id_to_category_map = {}
customer_purchase_power = train1.groupby("User_ID")["Purchase"].sum()
#customer_purchase_power.median()

values = customer_purchase_power.iteritems()
for key, val in values:
    if val <= 143662.0:
        user_id_to_category_map[key] = 1
    elif val <= 202156.0:
        user_id_to_category_map[key] = 2
    elif val <= 275408.0:
        user_id_to_category_map[key] = 3
    elif val <= 378451.0:
        user_id_to_category_map[key] = 4
    elif val <= 513874.0:
        user_id_to_category_map[key] = 5
    elif val <= 687241.0:
        user_id_to_category_map[key] = 6
    elif val <= 934097.0:
        user_id_to_category_map[key] = 7
    elif val <= 1332584.0:
        user_id_to_category_map[key] = 8
    elif val <= 2029891.0:
        user_id_to_category_map[key] = 9
    else:
        user_id_to_category_map[key] = 10
customer_purchase_power.median()
def get_customer_category(user_id):
    if user_id in user_id_to_category_map:
        return user_id_to_category_map[user_id]
    return 5
train1["Purchasing_power"] = list(map(lambda user_id: get_customer_category(user_id), train1["User_ID"]))
test1["Purchasing_power"] = list(map(lambda user_id: get_customer_category(user_id), test1["User_ID"]))
#test1.head()
train1 = train1.append(test1,sort=False,ignore_index=True)
train1.fillna(-999,inplace=True)

#OneHot Encoding of City Category - 
train1=pd.get_dummies(train1, columns=['City_Category'])
train1['Product_Category_2'] = train1['Product_Category_2'].astype(int)
train1['Product_Category_3'] = train1['Product_Category_3'].astype(int)

In [2]:
train=train1[:547316]
test=train1[547316:]
test=test.drop(['Purchase'],1)

In [3]:
train["UID_Min"] = train.groupby(['User_ID'])['Purchase'].transform('min')
userID_min_dict = train.groupby(['User_ID'])['Purchase'].min().to_dict()
test['UID_Min'] = test['User_ID'].apply(lambda x:userID_min_dict.get(x,0))

train["UID_MaxP"] = train.groupby(['User_ID'])['Purchase'].transform('max')
userID_max_dict = train.groupby(['User_ID'])['Purchase'].max().to_dict()
test['UID_MaxP'] = test['User_ID'].apply(lambda x:userID_max_dict.get(x,0))

train["PID_MinP"] = train.groupby(['Product_ID'])['Purchase'].transform('min')
productID_min_dict = train.groupby(['Product_ID'])['Purchase'].min().to_dict()
test['PID_MinP'] = test['Product_ID'].apply(lambda x:productID_min_dict.get(x,0))

train["PID_MaxP"] = train.groupby(['Product_ID'])['Purchase'].transform('max')
productID_max_dict = train.groupby(['Product_ID'])['Purchase'].max().to_dict()
test['PID_MaxP'] = test['Product_ID'].apply(lambda x:productID_max_dict.get(x,0))

train["User_ID_Count"] = train.groupby(['User_ID'])['User_ID'].transform('count')
uID_count = train.groupby(['User_ID']).size().to_dict()
test['User_ID_Count'] = test['User_ID'].apply(lambda x:uID_count.get(x,0))

train["Product_ID_Count"] = train.groupby(['Product_ID'])['Product_ID'].transform('count')
pID_count = train.groupby(['Product_ID']).size().to_dict()
test['Product_ID_Count'] = test['Product_ID'].apply(lambda x:pID_count.get(x,0))

train["UID_MeanP"] = train.groupby(['User_ID'])['Purchase'].transform('mean')
userID_mean_dict = train.groupby(['User_ID'])['Purchase'].mean().to_dict()
test['UID_MeanP'] = test['User_ID'].apply(lambda x:userID_mean_dict.get(x,0))

train["PID_MeanP"] = train.groupby(['Product_ID'])['Purchase'].transform('mean')
productID_mean_dict = train.groupby(['Product_ID'])['Purchase'].mean().to_dict()
test['PID_MeanP'] = test['Product_ID'].apply(lambda x:productID_mean_dict.get(x,0))

train["Pc1_MeanP"] = train.groupby(['Product_Category_1'])['Purchase'].transform('mean')
pc1_mean_dict = train.groupby(['Product_Category_1'])['Purchase'].mean().to_dict()
test['Pc1_MeanP'] = test['Product_Category_1'].apply(lambda x:pc1_mean_dict.get(x,0))

train["Pc1_MinP"] = train.groupby(['Product_Category_1'])['Purchase'].transform('min')
pc1_min_dict = train.groupby(['Product_Category_1'])['Purchase'].min().to_dict()
test['Pc1_MinP'] = test['Product_Category_1'].apply(lambda x:pc1_min_dict.get(x,0))

train["Pc1_MaxP"] = train.groupby(['Product_Category_1'])['Purchase'].transform('max')
pc1_max_dict = train.groupby(['Product_Category_1'])['Purchase'].max().to_dict()
test['Pc1_MaxP'] = test['Product_Category_1'].apply(lambda x:pc1_max_dict.get(x,0))

train["Pc2_MeanP"] = train.groupby(['Product_Category_2'])['Purchase'].transform('mean')
pc2_mean_dict = train.groupby(['Product_Category_2'])['Purchase'].mean().to_dict()
test['Pc2_MeanP'] = test['Product_Category_2'].apply(lambda x:pc2_mean_dict.get(x,0))

train["Pc2_MinP"] = train.groupby(['Product_Category_2'])['Purchase'].transform('min')
pc2_min_dict = train.groupby(['Product_Category_2'])['Purchase'].min().to_dict()
test['Pc2_MinP'] = test['Product_Category_2'].apply(lambda x:pc2_min_dict.get(x,0))

train["Pc2_MaxP"] = train.groupby(['Product_Category_2'])['Purchase'].transform('max')
pc2_max_dict = train.groupby(['Product_Category_2'])['Purchase'].max().to_dict()
test['Pc2_MaxP'] = test['Product_Category_2'].apply(lambda x:pc2_max_dict.get(x,0))

train["Pc3_MeanP"] = train.groupby(['Product_Category_3'])['Purchase'].transform('mean')
pc3_mean_dict = train.groupby(['Product_Category_3'])['Purchase'].mean().to_dict()
test['Pc3_MeanP'] = test['Product_Category_3'].apply(lambda x:pc3_mean_dict.get(x,0))

train["Pc3_MinP"] = train.groupby(['Product_Category_3'])['Purchase'].transform('min')
pc3_min_dict = train.groupby(['Product_Category_3'])['Purchase'].min().to_dict()
test['Pc3_MinP'] = test['Product_Category_3'].apply(lambda x:pc3_min_dict.get(x,0))

train["Pc3_MaxP"] = train.groupby(['Product_Category_3'])['Purchase'].transform('max')
pc3_max_dict = train.groupby(['Product_Category_3'])['Purchase'].max().to_dict()
test['Pc3_MaxP'] = test['Product_Category_3'].apply(lambda x:pc3_max_dict.get(x,0))

userID_25p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['UID_25Perc'] = train['User_ID'].apply(lambda x:userID_25p_dict.get(x,0))
test['UID_25Perc'] = test['User_ID'].apply(lambda x:userID_25p_dict.get(x,0))

userID_50p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,50.)).to_dict()
train['UID_50Perc'] = train['User_ID'].apply(lambda x:userID_50p_dict.get(x,0))
test['UID_50Perc'] = test['User_ID'].apply(lambda x:userID_50p_dict.get(x,0))

userID_75p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['UID_75Perc'] = train['User_ID'].apply(lambda x:userID_75p_dict.get(x,0))
test['UID_75Perc'] = test['User_ID'].apply(lambda x:userID_75p_dict.get(x,0))

productID_25p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['PID_25Perc'] = train['Product_ID'].apply(lambda x:productID_25p_dict.get(x,0))
test['PID_25Perc'] = test['Product_ID'].apply(lambda x:productID_25p_dict.get(x,0))

productID_50p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['PID_50Perc'] = train['Product_ID'].apply(lambda x:productID_50p_dict.get(x,0))
test['PID_50Perc'] = test['Product_ID'].apply(lambda x:productID_50p_dict.get(x,0))

productID_75p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['PID_75Perc'] = train['Product_ID'].apply(lambda x:productID_75p_dict.get(x,0))
test['PID_75Perc'] = test['Product_ID'].apply(lambda x:productID_75p_dict.get(x,0))

pc1_25p_dict = train.groupby(['Product_Category_1'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Pc1_25Perc'] = train['Product_Category_1'].apply(lambda x:pc1_25p_dict.get(x,0))
test['Pc1_25Perc'] = test['Product_Category_1'].apply(lambda x:pc1_25p_dict.get(x,0))

pc1_50p_dict = train.groupby(['Product_Category_1'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Pc1_50Perc'] = train['Product_Category_1'].apply(lambda x:pc1_50p_dict.get(x,0))
test['Pc1_50Perc'] = test['Product_Category_1'].apply(lambda x:pc1_50p_dict.get(x,0))

pc1_75p_dict = train.groupby(['Product_Category_1'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Pc1_75Perc'] = train['Product_Category_1'].apply(lambda x:pc1_75p_dict.get(x,0))
test['Pc1_75Perc'] = test['Product_Category_1'].apply(lambda x:pc1_75p_dict.get(x,0))


pc2_25p_dict = train.groupby(['Product_Category_2'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Pc2_25Perc'] = train['Product_Category_2'].apply(lambda x:pc2_25p_dict.get(x,0))
test['Pc2_25Perc'] = test['Product_Category_2'].apply(lambda x:pc2_25p_dict.get(x,0))

pc2_50p_dict = train.groupby(['Product_Category_2'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Pc2_50Perc'] = train['Product_Category_2'].apply(lambda x:pc2_50p_dict.get(x,0))
test['Pc2_50Perc'] = test['Product_Category_2'].apply(lambda x:pc2_50p_dict.get(x,0))

pc2_75p_dict = train.groupby(['Product_Category_2'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Pc2_75Perc'] = train['Product_Category_2'].apply(lambda x:pc2_75p_dict.get(x,0))
test['Pc2_75Perc'] = test['Product_Category_2'].apply(lambda x:pc2_75p_dict.get(x,0))

pc3_25p_dict = train.groupby(['Product_Category_3'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Pc3_25Perc'] = train['Product_Category_3'].apply(lambda x:pc3_25p_dict.get(x,0))
test['Pc3_25Perc'] = test['Product_Category_3'].apply(lambda x:pc3_25p_dict.get(x,0))

pc3_50p_dict = train.groupby(['Product_Category_3'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Pc3_50Perc'] = train['Product_Category_3'].apply(lambda x:pc3_50p_dict.get(x,0))
test['Pc3_50Perc'] = test['Product_Category_3'].apply(lambda x:pc3_50p_dict.get(x,0))

pc3_75p_dict = train.groupby(['Product_Category_3'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Pc3_75Perc'] = train['Product_Category_3'].apply(lambda x:pc3_75p_dict.get(x,0))
test['Pc3_75Perc'] = test['Product_Category_3'].apply(lambda x:pc3_75p_dict.get(x,0))

train["Occupation_Count"] = train.groupby(['Occupation'])['Occupation'].transform('count')
occupation_count_dict = train.groupby(['Occupation']).size().to_dict()
test['Occupation_Count'] = test['Occupation'].apply(lambda x:occupation_count_dict.get(x,0))

train["Age_Count"] = train.groupby(['Age'])['Age'].transform('count')
age_count_dict = train.groupby(['Age']).size().to_dict()
test['Age_Count'] = test['Age'].apply(lambda x:age_count_dict.get(x,0))

train["Product_Category_1_Count"] = train.groupby(['Product_Category_1'])['Product_Category_1'].transform('count')
pc1_count_dict = train.groupby(['Product_Category_1']).size().to_dict()
test['Product_Category_1_Count'] = test['Product_Category_1'].apply(lambda x:pc1_count_dict.get(x,0))

train["Product_Category_2_Count"] = train.groupby(['Product_Category_2'])['Product_Category_2'].transform('count')
pc2_count_dict = train.groupby(['Product_Category_2']).size().to_dict()
test['Product_Category_2_Count'] = test['Product_Category_2'].apply(lambda x:pc2_count_dict.get(x,0))

train["Product_Category_3_Count"] = train.groupby(['Product_Category_3'])['Product_Category_3'].transform('count')
pc3_count_dict = train.groupby(['Product_Category_3']).size().to_dict()
test['Product_Category_3_Count'] = test['Product_Category_3'].apply(lambda x:pc3_count_dict.get(x,0))

<ipython-input-3-64a2cb926dd5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["UID_Min"] = train.groupby(['User_ID'])['Purchase'].transform('min')
<ipython-input-3-64a2cb926dd5>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["UID_MaxP"] = train.groupby(['User_ID'])['Purchase'].transform('max')
<ipython-input-3-64a2cb926dd5>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

<ipython-input-3-64a2cb926dd5>:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['PID_50Perc'] = train['Product_ID'].apply(lambda x:productID_50p_dict.get(x,0))
<ipython-input-3-64a2cb926dd5>:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['PID_75Perc'] = train['Product_ID'].apply(lambda x:productID_75p_dict.get(x,0))
<ipython-input-3-64a2cb926dd5>:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

In [4]:
gender_dict = {'F':0, 'M':1}
age_dict = {'0-17':0, '18-25':1, '26-35':2, '36-45':3, '46-50':4, '51-55':5, '55+':6}
stay_dict = {'0':0, '1':1, '2':2, '3':3, '4+':4}

train["Gender"] = train["Gender"].apply(lambda x: gender_dict[x])
test["Gender"] = test["Gender"].apply(lambda x: gender_dict[x])

train["Age"] = train["Age"].apply(lambda x: age_dict[x])
test["Age"] = test["Age"].apply(lambda x: age_dict[x])

train["Stay_In_Current_City_Years"] = train["Stay_In_Current_City_Years"].apply(lambda x: stay_dict[x])
test["Stay_In_Current_City_Years"] = test["Stay_In_Current_City_Years"].apply(lambda x: stay_dict[x])

<ipython-input-4-257936750274>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Gender"] = train["Gender"].apply(lambda x: gender_dict[x])
<ipython-input-4-257936750274>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["Age"] = train["Age"].apply(lambda x: age_dict[x])
<ipython-input-4-257936750274>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [5]:
xg1 = pd.DataFrame()
xg1['User_ID'] = test['User_ID']
xg1['Product_ID'] = test['Product_ID']
cat_col=["User_ID", "Product_ID"]
for var in cat_col:
    lb = preprocessing.LabelEncoder()
    full_var_data = pd.concat((train[var], test[var]), axis=0).astype('str')
    lb.fit(full_var_data)
    train[var] = lb.transform(train[var].astype('str'))
    test[var] = lb.transform(test[var].astype('str'))
train_y = train["Purchase"]
train.drop(["Purchase"], axis=1, inplace=True)
train.dtypes

<ipython-input-5-1c3fd29121bb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[var] = lb.transform(train[var].astype('str'))
C:\Users\tsans\anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


User_ID                         int32
Product_ID                      int32
Gender                          int64
Age                             int64
Occupation                      int64
Stay_In_Current_City_Years      int64
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2              int32
Product_Category_3              int32
Purchasing_power                int64
City_Category_A                 uint8
City_Category_B                 uint8
City_Category_C                 uint8
UID_Min                         int64
UID_MaxP                        int64
PID_MinP                        int64
PID_MaxP                        int64
User_ID_Count                   int64
Product_ID_Count                int64
UID_MeanP                     float64
PID_MeanP                     float64
Pc1_MeanP                     float64
Pc1_MinP                        int64
Pc1_MaxP                        int64
Pc2_MeanP                     float64
Pc2_MinP    

In [6]:
from sklearn.linear_model import LinearRegression
linRegr = LinearRegression()
linRegr.fit(train, train_y)
print('done fitting')
predictions = linRegr.predict(test)

done fitting


In [7]:
xg1['Purchase']=predictions
sum(n < 30 for n in xg1["Purchase"].values.flatten())
xg1[xg1["Purchase"] < 0] = 0
xg1.to_csv('linearfinal.csv')

In [8]:
print('Your score for this submission is : 2610.3038028637015')

Your score for this submission is : 2610.3038028637015
